The following is the BERT_RNN_preprocess

In [1]:
import pandas as pd
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import re
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

I0412 02:55:31.275488 140346962626304 file_utils.py:41] PyTorch version 1.4.0 available.
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda

In [25]:

    
def inputid (inputsent,tokenizername):
    tokenizer = BertTokenizer.from_pretrained(tokenizername)
    input_ids = []
    for sent in tqdm(inputsent):
        sent= word_tokenize(sent)[0:500]
        encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
        input_ids.append(encoded_sent)
    return input_ids

def maxwordnum(allsec):
    allsentlen=[]
    for i in tqdm(allsec):
        wordnu=len(i)
        allsentlen.append(wordnu)
    maxnum=max(np.array(allsentlen))
    return maxnum

def dxseqpadding (seq,maxnu):
    seq2=[]
    for i in tqdm(seq):
        stamp=len(i)
        i=np.pad(i,((0,maxnu-stamp)),'constant',constant_values=0)
        seq2.append(i)
    return seq2

def attid (inputsent):
    attention_masks = []
    for sent in tqdm(inputsent):
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)
    return attention_masks

def dataloader (trainval, test,args):
    train_inputs=trainval[0]
    train_inputs = torch.tensor(train_inputs)
    train_labels=trainval[1]
    train_labels = torch.tensor(train_labels)
    train_masks=trainval[2]
    train_masks = torch.tensor(train_masks)
    
    val_inputs=trainval[3]
    val_inputs = torch.tensor(val_inputs)
    val_labels=trainval[4]
    val_labels = torch.tensor(val_labels)
    val_masks=trainval[5]
    val_masks = torch.tensor(val_masks)
    
    test_inputs=test[0]
    test_inputs = torch.tensor(test_inputs)
    test_labels=test[1]
    test_labels = torch.tensor(test_labels)
    test_masks=test[2]
    test_masks = torch.tensor(test_masks)
    train_data = TensorDataset(train_inputs, train_masks, train_labels)    
    train_dataloader = DataLoader(train_data, batch_size=args, shuffle=True)
    
    validation_data = TensorDataset(val_inputs, val_masks, val_labels)    
    validation_dataloader = DataLoader(validation_data, batch_size=args, shuffle=True)
        
    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_dataloader = DataLoader(test_data, batch_size=args, shuffle=True)
    
    return (train_dataloader,validation_dataloader,test_dataloader)
    
def bertrnn_process (args,trainsent,valsent,testsent,trainlabel,vallabel,testlabel):
    if args['science']==True:       
        trainsci=inputid(trainsent,args['modelname2'])
        valsci=inputid(valsent,args['modelname2'])
        testsci=inputid(testsent,args['modelname2'])
        trainnor=inputid(trainsent,args['modelname1'])
        valnor=inputid(valsent,args['modelname1'])
        testnor=inputid(testsent,args['modelname1'])
        maxnum=maxwordnum(testnor)
        trainsci=dxseqpadding(trainsci,maxnum)
        valsci=dxseqpadding(valsci,maxnum)
        testsci=dxseqpadding(testsci,maxnum)
        trainnor=dxseqpadding(trainnor,maxnum)
        valnor=dxseqpadding(valnor,maxnum)
        testnor=dxseqpadding(testnor,maxnum)
        trainsciatt=attid(trainsci)
        valsciatt=attid(valsci)
        testsciatt=attid(testsci)
        trainnoratt=attid(trainnor)
        valnoratt=attid(valnor)
        testnoratt=attid(testnor)
        nortrainval=(trainnor,trainlabel,trainnoratt,valnor,vallabel,valnoratt)
        scitrainval=(trainsci,trainlabel,trainsciatt,valsci,vallabel,valsciatt)
        scitest=(testsci,testlabel,testsciatt)
        nortest=(testnor,testlabel,testnoratt)
        norloder=dataloader (nortrainval, nortest,int(args['batch_size']))
        sciloder=dataloader (scitrainval, scitest,int(args['batch_size']))
    else : 
            
        trainnor=inputid(trainsent,args['modelname1'])
        valnor=inputid(valsent,args['modelname1'])
        testnor=inputid(testsent,args['modelname1'])
        maxnum=maxwordnum(testnor)
        trainnor=dxseqpadding(trainnor,maxnum)
        valnor=dxseqpadding(valnor,maxnum)
        testnor=dxseqpadding(testnor,maxnum)
        trainnoratt=attid(trainnor)
        valnoratt=attid(valnor)
        testnoratt=attid(testnor)
        nortrainval=(trainnor,trainlabel,trainnoratt,valnor,vallabel,valnoratt)        
        nortest=(testnor,testlabel,testnoratt)
        norloder=dataloader (nortrainval, nortest,int(args['batch_size']))       
        sciloder=[]
    return norloder,sciloder

In [3]:
input_directory='../snorkel-pseudo-label/'

In [4]:
with open(input_directory+'keylist.txt', "r") as f:
    alist =f.read().splitlines()
    for line in alist:
        keylist=line.split(',')

In [5]:
trainsent=pd.read_csv(input_directory+'trainsent.csv').sent.values
valsent=pd.read_csv(input_directory+'valsent.csv').sent.values
testsent=pd.read_csv(input_directory+'testsent.csv').sent.values
trainlabel=pd.read_csv(input_directory+'trainlabel.csv')[keylist].values
vallabel=pd.read_csv(input_directory+'vallabel.csv')[keylist].values
testlabel=pd.read_csv(input_directory+'testlabel.csv').newpid.values

In [6]:
startw=np.count_nonzero(trainlabel== 1, axis=0)
defualt=np.max(startw)
squarer = lambda t: defualt/t
trainweight=np.array([squarer(xi) for xi in startw])

In [9]:
args={"modelname1":'bert-base-uncased',
      "modelname2":'allenai/scibert_scivocab_uncased',
      "num_labels":len(trainlabel[0]),
      "hidden_size":256,
      "num_layers":1,
      "bidirectional":1,
      "dropout":0.2,
      "batch_size":10,
      "epochs":100,
      "lr":0.001,
      "seed": 1,
      "early_stop_times":5,
      "science":True
}

In [26]:
norloder,sciloder=bertrnn_process(args,trainsent,valsent,testsent,trainlabel,vallabel,testlabel)

I0412 03:42:02.036038 140346962626304 tokenization_utils.py:420] Model name 'allenai/scibert_scivocab_uncased' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'allenai/scibert_scivocab_uncased' is a path, a model identifier, or url to a directory containing tokenizer files.
I0412 03:42:02.928238 140346962626304 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/allenai/scibert_scivocab_uncased/vocab.txt from cache at

The following is model and running code

In [22]:
import torch.nn as nn
import torch
from transformers import BertModel
from torch.optim import Adam
from torch.autograd import Variable
from sklearn.metrics import label_ranking_loss
from nltk.tokenize import sent_tokenize,word_tokenize
from transformers import BertModel, BertConfig, AdamW
from tqdm import tqdm

class bert_rnn(nn.Module):
    def __init__(self, args):
        super(bert_rnn, self).__init__()
        self.args = args
        self.emb1=BertModel.from_pretrained(self.args['modelname1'],num_labels = self.args['num_labels'],output_attentions = False,output_hidden_states = False)#.cuda(3)
        self.emb1_size=self.emb1.config.hidden_size
        if self.args['science']==True:
            self.emb2=BertModel.from_pretrained(self.args['modelname2'],num_labels = self.args['num_labels'],output_attentions = False,output_hidden_states = False)#.cuda(3)
            self.emb2_size=self.emb2.config.hidden_size
            self.emb_size=self.emb1_size+self.emb2_size
        else:
            self.emb_size=self.emb1_size
        self.lin1 = nn.Linear(self.emb_size, self.args['hidden_size'])
        if self.args['bidirectional']>1:
            bidirectional=2
            dif=True
        else:
            bidirectional=1
            dif=False
        self.rnn=nn.LSTM(input_size=self.args['hidden_size'], hidden_size =self.args['hidden_size'], num_layers =self.args['num_layers'],batch_first= True,dropout=self.args['dropout'],bidirectional=dif)
        self.lin2 = nn.Linear(self.args['hidden_size'], self.args['num_labels'])
        self.sigmoid=nn.Sigmoid()
    def forward(self,data1,mask1,target,data2=None,mask2=None):
        if self.args['science']==True:
            emb1=self.emb1(data1,attention_mask=mask1)
            last_hidden_states1 = emb1[0]
            emb2=self.emb2(data2,attention_mask=mask2)
            last_hidden_states2 = emb2[0]
            last_hidden_states = torch.cat((last_hidden_states1, last_hidden_states2), 2)
            last_hidden_states = self.lin1(last_hidden_states)
        else:
            emb1=self.emb1(data1)
            last_hidden_states1 = emb1[0]
            last_hidden_states = self.lin1(last_hidden_states1)
        output, _ = self.rnn(last_hidden_states)#,# (self.h0, self.c0))
        output=torch.mean(output, 1)
        out = self.lin2(output)
        #out=self.sigmoid(out)
        return out,target
#     def parameters (self):
#         if self.args['science']==True:
#             params=list(self.emb1.parameters())+list(self.emb2.parameters())+list(self.rnn.parameters())+list(self.lin1.parameters())+list(self.lin2.parameters())
#         else:
#             params=list(self.emb1.parameters())+list(self.rnn.parameters())+list(self.lin1.parameters())+list(self.lin2.parameters())
#         return params

In [27]:
def train_(model,loss,optimizer,dataloaders1,epoch,dataloaders2=None):
    allloss=[]
    allbatch=[]
    print('Train')
    model.train()
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            optimizer.zero_grad()
            out,target = model.forward(data1,mask1,target1,data2,mask2)
            lossall = loss(out,target.float())            
            lossall = torch.sum(lossall)
            lossall.backward()
            optimizer.step()
            loss1=lossall.item()
            allloss.append(loss1)
            allbatch.append(batch_idx*epoch)
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda(3)
            data1 = Variable(data1).cuda(3)
            mask1=Variable(mask1).cuda(3)
            optimizer.zero_grad()
            out,target = model.forward(data1,mask1,target1)
            lossall = loss(out,target.float())            
            lossall = torch.sum(lossall)
            lossall.backward()
            optimizer.step()
            allloss.append(lossall)
            allbatch.append(batch_idx*epoch)
    return allloss, allbatch
def val_(model,dataloaders1,dataloaders2=None):
    model.eval()
    allout=[]
    alltarget=[]
    print('Validation')
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1,data2,mask2)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(list(out))
            alltarget.append(list(target))
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(list(out))
            alltarget.append(list(target))
    return allout,alltarget
def test_(model,dataloaders1,dataloaders2=None):
    model.eval()
    allout=[]
    alltarget=[]
    print('test')
    if args['science']==True:
        for batch_idx, batch in tqdm(enumerate(zip(dataloaders1, dataloaders2))):
            data1, mask1, target1 = batch[0]
            data2, mask2, target2 = batch[1]
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            target2 = Variable(target2).cuda()
            data2 = Variable(data2).cuda()
            mask2=Variable(mask2).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1,data2,mask2)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(out)
            alltarget.append(target)
    else:
        for batch_idx, batch in tqdm(enumerate(dataloaders1)):
            data1, mask1, target1 = batch
            target1 = Variable(target1).cuda()
            data1 = Variable(data1).cuda()
            mask1=Variable(mask1).cuda()
            with torch.no_grad(): 
                out,target = model.forward(data1,mask1,target1)
            out=out.cpu().detach().numpy()
            target=target.to('cpu').numpy().astype(int)
            
            allout.append(out)
            alltarget.append(target)
    return allout,alltarget
def result(result):
    alllist=[]
    for i in result:
        for j in i:
            alllist.append(j)
    return alllist


In [28]:
device = 3#("cuda" if torch.cuda.is_available() else "cpu" )
torch.cuda.set_device(device)
model= bert_rnn(args)
#if torch.cuda.device_count() > 1:
#    model=nn.DataParallel(model, device_ids=[0,1,2,4,5])
model.to(device)
params=model.parameters()
optimizer = AdamW(params,lr = 2e-5, eps = 1e-8 )
trainweight=torch.tensor(trainweight).float().cuda()
loss = nn.BCEWithLogitsLoss(pos_weight=trainweight)
#loss = nn.MSELoss()
alloss=[]
allbatch=[]
dev_lrl=1
vallrl=[]
testpred=[]
testpid=[]
current_early_stop_times=0
for epoch in range(1, args['epochs'] + 1):
    epochloss, epochbatch=train_(model,loss,optimizer,norloder[0],epoch,dataloaders2=sciloder[0])
    alloss.append(epochloss)
    allbatch.append(epochbatch)
    allout,alltarget=val_(model,norloder[1],sciloder[1])
    allout1=result(allout)
    alltarget1=result(alltarget)
    epochlrl=label_ranking_loss(alltarget1,allout1)
    vallrl.append(epochlrl)
    if epochlrl < dev_lrl:
        print("- new best lrl{}".format(epochlrl))
#         allout,alltarget=test_(model,norloder[2],sciloder[2])
#         allout=result(allout)
#         alltarget=result(alltarget)
#         allpd=pd.DataFrame(allout,columns=keylist)
#         allpd['newpid']=alltarget
#         allpd.to_csv('Bert_NIH_task4_Q1_LSTM.csv')
        dev_lrl = epochlrl
        current_early_stop_times = 0
    else:
        current_early_stop_times += 1
        print(current_early_stop_times)
    if current_early_stop_times >= args['early_stop_times'] :
        break;
print ("- early stopping {} epochs without improvement".format(epoch))

I0412 03:42:23.766776 140346962626304 configuration_utils.py:283] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0412 03:42:23.769320 140346962626304 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 8,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initiali

Train


66it [00:18,  3.48it/s]
2it [00:00, 12.54it/s]

Validation


17it [00:01, 12.82it/s]
0it [00:00, ?it/s]

...

4
Train


66it [00:18,  3.58it/s]
2it [00:00, 12.57it/s]

Validation


17it [00:01, 12.81it/s]

5
- early stopping 23 epochs without improvement
